In [1]:
import pandas as pd
import re
import time
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from selenium import webdriver
from textacy.preprocess import preprocess_text
import nltk

from bs4 import BeautifulSoup as bs

In [2]:
start = time.time()

wd = webdriver.Chrome()

b_summary = []
b_detail = []
b_date = []
b_summary_clean = []
b_detail_clean = []

for i in range(1,2):
    print 'scrapping page : ', i
    url = 'http://thehungrygeek.com/category/food/singapore-food/page/' + str(i) + '/'   

    try:
        wd.get(url)
    except:
        continue
    response = wd.page_source
    soup = bs(response, 'html.parser')
    
    try:
        blogs = soup.find_all('h2', {'class' : 'entry-title'})
    except:
        continue
        
    for summary in blogs:
        try:
            href = summary.find('a').get('href')
            print 'href : ', href
        except:
            continue            
        
        try:
            wd.get(href)
        except:
            continue
        
        r = wd.page_source
        s = bs(r, 'html.parser')

        # find date
        try:
            date = s.find('time', {'class' : 'entry-date'}).text
        except:
            continue
            
        # find detail
        try:
            y = s.find('section', {'class' : "entry-content"}).find_all('p')
        except:
            continue
        doc = []
        for p in y:
            if p.text:
                doc.append(p.text)
        d = ' '.join(doc)

        b_summary.append(summary.text)
        b_detail.append(d)
        b_date.append(date)
        
        # clean summary and detail 
        clean_summary = preprocess_text(summary.text, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        clean_detail = preprocess_text(d, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        b_summary_clean.append(clean_summary)
        b_detail_clean.append(clean_detail)
        
        print 'cleaned detail snippet : ', clean_detail[:75]
        
end = time.time()
exe_time = (end - start)/60
print 'Time taken :',(exe_time),' minutes'

print len(b_date)

scrapping page :  1
href :  http://thehungrygeek.com/2017/10/29/pan-pacific-orchard-10-at-claymore-weekday-lunch-buffet/
cleaned detail snippet :  the entrance of number at claymore pan pacific orchard singapore is a very 
href :  http://thehungrygeek.com/2017/10/07/open-door-policy/
cleaned detail snippet :  storefront of open door policy if youre searching for a nice bistro for a h
href :  http://thehungrygeek.com/2017/09/19/49-seats-super-delicious-western-food/
cleaned detail snippet :  the storefront of number seats here are the highlights number seats is a de
href :  http://thehungrygeek.com/2017/09/19/matsuya-sushi-bar/
cleaned detail snippet :  fancy of an exquisite japanese meal at an affordable price matsuya sushi ba
href :  http://thehungrygeek.com/2017/08/26/provisions-dempsey-skewers-claypot-rice-bar/
cleaned detail snippet :  the storefront of provisions dempsey dempsey hill is the place to go for th
href :  http://thehungrygeek.com/2017/06/28/the-masses-singapore/
cleane

In [3]:
# remove stop words from cleaned summary and cleaned detail and lemmatize them

stoplist = set(stopwords.words('english') + \
               ['number',"singapore", "food", "im", "street", 
                'porn','watch', 'video', 'centre', '...',
                'eat', 'day', 'time', 'cdataadsbygoogle', 'windowadsbygoogle',
                'wa', 'ha', 'come', 'place', 'dish', 'bring', 'think', 'quite','located',
                'month', 'went', 'probably', 'am', 'pm', 'say', 'said','including','year','item',
                'youre', 'sure', 'dont', 'came','really', 'got', 'thing', 'address', 'photo',
                'credit', 'opening', 'hour'] \
                 + list(ENGLISH_STOP_WORDS))

In [4]:
start = time.time()

summary_tokens = []
for item in b_summary_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    summary_tokens.append(t)
    
detail_tokens = []
for item in b_detail_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    detail_tokens.append(t)   

    
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'
    
print len(summary_tokens)
print len(detail_tokens)

Time taken : 0.245280981064  seconds
15
15


In [5]:
start = time.time()

summary_lem =[]
wnl = WordNetLemmatizer()
for item in summary_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    summary_lem.append(t) 
    
detail_lem =[]
wnl = WordNetLemmatizer()
for item in detail_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    detail_lem.append(t)     
   
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'

print len(summary_lem)
print len(detail_lem)

Time taken : 4.2247030735  seconds
15
15


In [6]:
# create final list to fit in dataframe
flist = []
for i in range(len(b_summary)):
    f = [b_summary[i], b_detail[i], b_date[i], summary_lem[i], detail_lem[i]]
    flist.append(f)

In [7]:
df = pd.DataFrame(flist, columns = ['Title', 'Details', 'Date', 'Processed Summary', 'Processed Detail'])  
df.tail()

,Title,Details,Date,Processed Summary,Processed Detail
10,Tai Cheong Bakery Cha Chaan Teng Opens @ Holla...,The storefront of Tai Cheong Bakery’s new rest...,6 Dec 2016,tai cheong bakery cha chaan teng open holland ...,storefront tai cheong bakery new restaurant ho...
11,Morganfield’s – These Are Some Really Deliciou...,The storefront of Morganfield’s at Star Vista ...,16 Nov 2016,morganfields delicious rib,storefront morganfields star vista morganfield...
12,Burp Kitchen & Bar – The Cafe with a View!,The storefront of Burp Kitchen & Bar Looking f...,14 Nov 2016,burp kitchen bar cafe view,storefront burp kitchen bar looking nice date ...
13,Sugar Lips – Dessert Colour Explosion @ Orchar...,The storefront of Sugar Lips! Sugar Lips is a ...,30 Sep 2016,sugar lip dessert colour explosion orchard gat...,storefront sugar lip sugar lip new dessert caf...
14,Craftholic Cafe – Super Cute Plushies from Japan!,The storefront of Craftholic Cafe Craftholic i...,20 Sep 2016,craftholic cafe super cute plushies japan,storefront craftholic cafe craftholic cutest h...


In [8]:
len(df)

15

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
len(df)

15

In [11]:
x = df.iloc[3,1]
print x

Fancy of an exquisite Japanese meal at an affordable price? Matsuya Sushi Bar might be the right place for you to have a true taste of Japan. Hidden along Seah Street in the Bugis area, this cozy sushi bar serves a wide array of Japanese dishes for both lunch and dinner. There’s no obvious signage in front of the restaurant and it could be easily missed when you pass by. We didn’t notice the eatery until we were invited down to try out their food, including an Omakase course. For the uninitiated, Omakase is a serving style in Japan where patrons will be entrusting the chef to serve them with his choices of food. Storefront of Matsuya Sushi Bar The biggest charm of Omakase is to be served with fresh food and be pleasantly surprised by the chef’s creativity in preparing his choices of food. The ingredients used at Matsuya Sushi Bar are directly imported from Japan. Save yourself the tickets to fly to Japan for an authentic taste of Japan. The Omakase course served to us includes appetize

In [12]:
# pickle dataframe
df.to_pickle('hungrygeek.pickle')

In [13]:
wd.quit()